In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import cavsim
from cavsim import Measure
from cavsim.connectors import BaseConnector, Connector
from cavsim.channels import ImportChannel, ExportChannel
from cavsim.components import BaseComponent, Component
from cavsim.solvers import BaseSolver, SimpleSolver
from cavsim.fluids import Fluid

In [3]:
from cavsim.pipes.pipe import Pipe
from cavsim.boundaries.left_boundary_pressure import LeftBoundaryPressure
from cavsim.boundaries.left_boundary_velocity import LeftBoundaryVelocity
from cavsim.boundaries.right_boundary_pressure import RightBoundaryPressure
from cavsim.boundaries.right_boundary_velocity import RightBoundaryVelocity
from cavsim.boundaries.simple_pipe_connector import PipeConnectorSimple
from cavsim.boundaries.zeta_joint import ZetaJoint
from cavsim.boundaries.simple_t_joint import SimpleTJoint
from cavsim.boundaries.simple_right_gasdampener import RightGasBubbleSimple
from cavsim.boundaries.frictionless_gasdampener import FrictionlessGasDampener
from cavsim.boundaries.gasdampener import GasDampener
from cavsim.boundaries.pump_with_valves_model_new import PumpSuctionValve
from cavsim.boundaries.reflectionless_left_boundary_condition import LeftBoundaryReflectionFree
from cavsim.boundaries.reflectionless_right_boundary_condition import RightBoundaryReflectionFree

In [4]:
fluid = Fluid(1000, 1e-3, 1.08e9, 2.3e-3,initial_pressure=128000)

In [5]:
pipe1 = Pipe(1.5, 1.5, 0.002, 2e11, 1e-7, 300)
pipe2 = Pipe(0.056, 4.5, 0.002, 2e11, 1e-7, 300)
pipe4 = Pipe(0.056, 0.5, 0.002, 2e11, 1e-7, 10)
pipe5 = Pipe(0.056, 0.5, 0.002, 2e11, 1e-7, 10)
pipe3 = Pipe(0.025, 28.7, 0.002, 2e11, 1e-7, 300, initial_pressure=20e5)

In [6]:
#lb1 = LeftBoundaryReflectionFree(0, 1.28e5)
lb1 = LeftBoundaryPressure(1.28e5)
pipe1.connect(lb1)

In [7]:
conns = ZetaJoint(1.0)
pipe1.connect(conns)
pipe2.connect(conns)

In [8]:
#conn = SimpleTJoint()
#pipe2.connect(conn)
#pipe4.connect(conn.right2)
#pipe5.connect(conn.right)

In [9]:
#rb1 = RightBoundaryReflectionFree(1.28e5, 0)
#rb1 = RightGasBubbleSimple(128000, 5e-6, 1.0)
#rb1.connect(pipe4)

In [10]:
rb = RightBoundaryPressure(20e5)
rb.connect(pipe3)

In [11]:
pump = PumpSuctionValve(suction_valve_density=7950.0,               # kg/m³
            suction_spring_force0 = 3.438,                  # N
            suction_spring_stiffness = 3438.0,            # N/m
            suction_spring_mass = 0.022,                 # kg
            suction_valve_mass = 0.068,                  # kg
            suction_outer_diameter = 0.040,              # m
            suction_inner_contact_diameter = 0.03644173,      # m
            suction_seat_tilt = 90/180*np.pi,                   # °
            suction_flow_constant_1 = 7.63e-3,             # -
            suction_flow_constant_2 = 6.480,             # -
            suction_friction_factor_a = 107.00,           # -
            suction_friction_factor_b = 74.00,           # -
            suction_friction_factor_c = 1.40,           # -
            suction_friction_factor_d = -2.40,           # -
            suction_factor_k0 = 0.50,                   # -
            suction_factor_k1 = 0.0,                   # -
            suction_factor_k2 = 0.0,                   # -
            suction_max_displacement = 25e-3,                   # m
            suction_outer_contact_diameter = 0.040,      # m
            discharge_valve_density = 7950.0,             # kg/m³
            discharge_spring_force0 = 3.438,             # N
            discharge_spring_stiffness = 3438.0,          # N/m
            discharge_spring_mass = 0.022,               # kg
            discharge_valve_mass = 0.068,                # kg
            discharge_outer_diameter = 0.040,            # m
            discharge_inner_contact_diameter = 0.03644173,    # m
            discharge_seat_tilt = 90/180*np.pi,                 # °
            discharge_flow_constant_1 = 7.63e-3,           # -
            discharge_flow_constant_2 = 6.480,           # -
            discharge_friction_factor_a = 107.00,         # -
            discharge_friction_factor_b = 74.00,         # -
            discharge_friction_factor_c = 1.40,         # -
            discharge_friction_factor_d = -2.40,         # -
            discharge_factor_k0 = 0.50,                 # -
            discharge_factor_k1 = 0.0,                 # -
            discharge_factor_k2 = 0.0,                 # -
            discharge_max_displacement = 25e-3,                 # m
            discharge_outer_contact_diameter = 0.040,    # m
            pump_radius = 0.010/2.0,                         # m
            rpm = 70.0,                                 # U/min
            rratio = 0.175,                              # -
            phi0 = 180,                                # °
            piston_diameter = 0.07,                     # m
            death_volume = 3.442416e-4)                        # m³)

In [12]:
pipe2.connect(pump)
pipe3.connect(pump)

In [13]:
plogs1 = []
plogs2 = []
plogs3 = []
vlogs = []
vlogs1 = []
flogs = []
rblogs = []
relogs = []
dislog = []
springlog = []
contact_pressurelog = []
velocity = []
pumplog = []
pump_speed = []
pump_value = []
flow_log = []
lower_pressure_log = []
contact_pressure_log = []
damping_log = []




def logging(time):
    plogs1.append(pipe1.field_wide_slice('pressure', 0) + 0.0)
    plogs2.append(pipe2.field_wide_slice('pressure', 0) + 0.0)
    plogs3.append(pipe3.field_wide_slice('pressure', 0) + 0.0)
    flogs.append(pipe2.field_wide_slice('friction_steady', 0) + 0.0)
    rblogs.append(rb._velocity[0,1])
    pumplog.append(pump.field_wide_slice('pump_pressure', 0) + 0.0)
    dislog.append(pump.field_wide_slice('suction_displacement', 0) + 0.0)
    relogs.append(pipe2.field_wide_slice('reynolds', 0) + 0.0)
    pump_speed.append(pump.field_wide_slice('discharge_pressure', 1) +0.0)
    pump_value.append(pump.field_wide_slice('discharge_displacement', 1) +0.0)
    springlog.append(pump.field_wide_slice('discharge_valve_velocity', 1) +0.0)
    contact_pressurelog.append(pump.field_wide_slice('suction_contact_pressure_force', 1) +0.0)
    flow_log.append(pump.field_wide_slice('suction_flow_force', 1) +0.0)
    velocity.append(pump.field_wide_slice('suction_valve_velocity', 1) +0.0)
    lower_pressure_log.append(pump.field_wide_slice('discharge_lower_pressure_force', 1) +0.0)
    contact_pressure_log.append(pump.field_wide_slice('discharge_contact_pressure_force', 1) +0.0)
    damping_log.append(pump.field_wide_slice('discharge_upper_pressure_force', 1) +0.0)

In [14]:
solver = SimpleSolver()
solver.fluid = fluid
solver.seeds = [pipe1, pipe2, pipe3]
solver._callback = logging

In [15]:
solver.solve(1e20, 3.5, 3)

 0:00:00 [                              |  0.00%]  2:30:37  Currently at time   0.000 of   3.500

l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (1.0776024615741177e-05 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x00000283B591FCF8>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))
l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (1.5435102963876856e-05 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x00000283B591FD68>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))
l:\cavsim\cavsim\base\solvers\simple_solver.py:46: UserWarning: Smaller timestep required by component! (9.479544404329514e-05 < 1e+20 by <cavsim.pipes.pipe.Pipe object at 0x00000283B59336D8>)
  warn('Smaller timestep required by component! ({} < {} by {})'.format(component_time, delta_t, component))


 0:00:56 [                              |  0.90%]  1:43:15  Currently at time   0.031 of   3.500-----------------------------------------
2411.819097907119
106.92767598568896
0.7716479833986767
3.438
0.0
2513.2741049935285
0.03149831995181018
1.9468859355171036e-09
 0:00:56 [                              |  0.90%]  1:43:16  Currently at time   0.032 of   3.500-----------------------------------------
2414.887863588923
106.97974093057782
0.7716480308399161
3.438
-1.4222502113795813e-06
2512.695367196564
0.03151987200104167
5.702708733903554e-09
 0:00:56 [                              |  0.90%]  1:43:16  Currently at time   0.032 of   3.500-----------------------------------------
2417.9586672513237
107.01878200083425
0.7716481264460435
3.438
-5.575768828251416e-06
2511.529063030634
0.03154142405027316
8.538369848766974e-09
 0:00:56 [                              |  0.90%]  1:43:16  Currently at time   0.032 of   3.500-----------------------------------------
2421.0315087801914
107.06301

 0:15:19 [===                           | 12.47%]  1:47:33  Currently at time   0.436 of   3.500-----------------------------------------
2423.225200964352
107.26157664325294
0.7716476422751307
3.438
-6.10241571909674e-06
2517.43547908861
0.436375116814544
7.544187364182992e-09
 0:15:19 [===                           | 12.47%]  1:47:33  Currently at time   0.436 of   3.500-----------------------------------------
2422.7378083863996
107.19600678961926
0.7716478459512012
3.438
-8.290705806871084e-06
2514.9508314877526
0.4363966688637755
9.062403949812211e-09
 0:15:19 [===                           | 12.47%]  1:47:32  Currently at time   0.436 of   3.500-----------------------------------------
2422.249071024303
107.13183431196182
0.7716480443712853
3.438
-1.086476015592679e-05
2512.5302973729313
0.43641822091300697
1.042304393205711e-08
 0:15:19 [===                           | 12.47%]  1:47:32  Currently at time   0.436 of   3.500-----------------------------------------
2421.7589888807

 0:15:20 [===                           | 12.49%]  1:47:31  Currently at time   0.437 of   3.500-----------------------------------------
2408.54484451432
105.88204458688061
0.7716515821580809
3.438
-8.247461863010317e-05
2469.371977694204
0.43700012624225715
3.184417198662399e-08
 0:15:20 [===                           | 12.49%]  1:47:31  Currently at time   0.437 of   3.500-----------------------------------------
2408.018454391776
105.84769888918422
0.7716516676970004
3.438
-8.592624363023269e-05
2468.3284506651876
0.43702167829148864
3.186116132181979e-08
 0:15:20 [===                           | 12.49%]  1:47:31  Currently at time   0.437 of   3.500-----------------------------------------
2407.4907195708543
105.81359777935758
0.7716517522294554
3.438
-8.600764944504877e-05
2467.297201149272
0.4370432303407201
3.256775042410206e-08
 0:15:20 [===                           | 12.49%]  1:47:31  Currently at time   0.437 of   3.500-----------------------------------------
2406.96164005

...

 1:51:04 [======================        | 74.09%]  0:38:50  Currently at time   2.593 of   3.500-----------------------------------------
2614.3742790083056
86.9740127115679
0.7717372953103145
3.438
-0.06375585352121808
1423.3167796993346
2.593271875828017
9.836229394287296e-07
 1:51:04 [======================        | 74.09%]  0:38:50  Currently at time   2.593 of   3.500-----------------------------------------
2618.057371687778
86.72624026968627
0.7717384948972573
3.438
-0.06422389751055428
1408.6711318398052
2.5932934278772484
1.0078819380415906e-06
 1:51:04 [======================        | 74.09%]  0:38:50  Currently at time   2.593 of   3.500-----------------------------------------
2621.7425436887265
86.4158264086575
0.7717399253390052
3.438
-0.06741502939656786
1391.2067920679515
2.59331497992648
1.0157503223627424e-06
 1:51:04 [======================        | 74.10%]  0:38:49  Currently at time   2.593 of   3.500-----------------------------------------
2625.4297944867917
86.1

 1:51:05 [======================        | 74.11%]  0:38:48  Currently at time   2.594 of   3.500-----------------------------------------
2722.027774518713
77.8296053161351
0.7717793564228751
3.438
-0.13752337349059204
909.7020780463184
2.59389688525573
1.4655714346737744e-06
 1:51:05 [======================        | 74.11%]  0:38:48  Currently at time   2.594 of   3.500-----------------------------------------
2725.7711274778235
77.43318128948079
0.7717811075258767
3.438
-0.14209158280090795
888.3148753287612
2.5939184373049615
1.479868289822805e-06
 1:51:05 [======================        | 74.11%]  0:38:48  Currently at time   2.594 of   3.500-----------------------------------------
2729.5165570059085
77.2601706009714
0.7717820362567605
3.438
-0.14486745515652882
876.9716275710068
2.593939989354193
1.4885674940823855e-06
 1:51:05 [======================        | 74.11%]  0:38:48  Currently at time   2.594 of   3.500-----------------------------------------
2733.264062771376
76.83762

 1:51:06 [======================        | 74.13%]  0:38:46  Currently at time   2.595 of   3.500-----------------------------------------
2835.2314043264964
66.22447692916612
0.7718308895105781
3.438
-0.2658757565931265
280.1585558668107
2.5945434467326747
2.027635670294392e-06
 1:51:07 [======================        | 74.13%]  0:38:46  Currently at time   2.595 of   3.500-----------------------------------------
2839.0370236264866
65.76728870359334
0.7718328682548384
3.438
-0.2713839557246767
255.97980488465575
2.594564998781906
2.0493288547237683e-06
 1:51:07 [======================        | 74.13%]  0:38:46  Currently at time   2.595 of   3.500-----------------------------------------
2842.844717343251
65.18169067534404
0.7718353196479203
3.438
-0.2772026587194901
226.02504712671478
2.5945865508311377
2.078785158102354e-06
 1:51:07 [======================        | 74.13%]  0:38:46  Currently at time   2.595 of   3.500-----------------------------------------
2846.654485887791
64.788

 1:51:08 [======================        | 74.15%]  0:38:45  Currently at time   2.595 of   3.500-----------------------------------------
2946.436265609988
53.90825932055039
0.7718898030260694
3.438
-0.45899254629326836
-439.9044012232311
2.595168456160388
2.6582038512047888e-06
 1:51:08 [======================        | 74.15%]  0:38:44  Currently at time   2.595 of   3.500-----------------------------------------
2950.302009722614
53.537178079537355
0.7718920474945039
3.438
-0.4656786063086325
-467.3446875502664
2.5951900082096193
2.6866874527686517e-06
 1:51:08 [======================        | 74.15%]  0:38:44  Currently at time   2.595 of   3.500-----------------------------------------
2954.169826427651
53.053598351151415
0.7718949034471325
3.438
-0.4756823571477229
-502.2616224665877
2.595211560258851
2.717258679621323e-06
 1:51:08 [======================        | 74.15%]  0:38:44  Currently at time   2.595 of   3.500-----------------------------------------
2958.039715831084
52.6

KeyboardInterrupt: 

In [16]:
pfield1 = np.stack(plogs1)
pfield2 = np.stack(plogs2)
pfield3 = np.stack(plogs3)
disfield = np.stack(dislog)
velfield = np.stack(velocity)
pressure = np.stack(pumplog)
pump_velocity = np.stack(pump_speed)
values = np.stack(pump_value)
flow_force = np.stack(flow_log)
spring_force = np.stack(springlog)
lower_pressure_force = np.stack(lower_pressure_log)
contact_pressure_force = np.stack(contact_pressure_log)
damping_force = np.stack(damping_log)

# print(pump._cases[val1:val])

In [18]:
time = np.linspace(0, 3.0, int(pfield1[:,-2].shape[0]))
#plt.figure(figsize=(16, 10))
fig, ax1 = plt.subplots(figsize=(16, 10))
#plt.subplot(1, 2, 1)
#plt.plot(vfield[:,0])
#plt.plot(1, 2, 2)
val1 = 0
val = -1
#val1 = 0
#val = -1

#upper_force = upperp_field[val1:val, 0] + gfield[val1:val, 0] + springforce[val1:val, 0]
#lower_force = lowp_field[val1:val, 0] + contact_pressure[val1:val, 0]
#result1 = lower_force - upper_force
#result2 = flow_field[val1:val, 0] - gfield[val1:val, 0] - springforce[val1:val, 0] - dampingfield[val1:val, 0]
#upper_flow = gfield[val1:val, 0] + springforce[val1:val, 0] + dampingfield[val1:val, 0]
#lower_flow = flow_field[val1:val, 0]
#ax1.plot(time[val1:val], upper_flow, color='red')
#ax1.plot(time[val1:val], lower_flow, color='blue')


#plt.subplot(1, 2, 2)
#ax1.plot(result1[val1:val])
#ax1.plot(result2[val1:val])
#ax1.plot(time[val1:val], upperp_field[val1:val, 0], color='red')
#ax1.plot(time[val1:val], lowp_field[val1:val, 0], color='green')
#ax1.plot(time[val1:val], dampingfield[val1:val, 0], color='blue')

#ax1.plot(time[val1:val], contact_pressure[val1:val, 0], color='black')
#ax1.plot(time[val1:val], springforce[val1:val, 0], color='grey')
#ax1.plot(time[val1:val], upper_force, color='red')
#ax1.plot(time[val1:val], lower_force, color='blue')

plt.plot(time[val1:val], pfield3[val1:val, 1], color='red')
plt.plot(time[val1:val], pump_velocity[val1:val, 0], color='black')
plt.plot(time[val1:val], pfield2[val1:val, -2], color='green')
plt.plot(time[val1:val], pressure[val1:val, 0],color='violet')


#ax1.plot(time[val1:val], deltap[val1:val, 0], color='green')

#plt.xlim(time[val1],time[val])
#plt.ylim(0e5, 3e5)

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Pressure', color=color)
ax1.set_ylim(0e5, 23e5)
#ax1.plot(time[val1:val], pfield1[val1:val, 1], color='red')
#ax1.plot(time[val1:val], pfield2[val1:val, -2], color='green')
ax1.tick_params(axis='y', labelcolor=color)


ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.plot(time[val1:val], disfield[val1:val, 0]*1000, color=color)  # we already handled the x-label with ax1
ax2.plot(time[val1:val], values[val1:val, 0]*1000, color = 'black')
#ax2.plot(time[val1:val], contact_pressure_force[val1:val, 0], color = 'green')
#ax2.plot(time[val1:val], spring_force[val1:val, 0], color = 'green')
#ax2.plot(time[val1:val], contact_pressure_force[val1:val, 0]+lower_pressure_force[val1:val, 0], color = 'black')
#ax2.plot(time[val1:val], flow_force[val1:val, 0]-spring_force[val1:val, 0], color = 'red')
#ax2.plot(time[val1:val], zeta[val1:val, 0], color='green')
#ax2.plot(time[val1:val], flow_field[val1:val, 0], color='black')
#ax2.tick_params(axis='y', labelcolor=color)
#ax2.set_ylim(0e5, 0.0001)

#plt.subplot(1, 2, 1)
#plt.plot(time[val1:val], disfield[val1:val, 0]*1000, color='black')
#plt.plot(time[val1:val], pump(time[val1:val]))
#plt.plot(time[val1:val], velfield[val1:val, 0], color='blue')
#plt.plot(time[val1:val], pfield2[val1:val, 1], color='red')
#pltplot(time[val1:val], pfield1[val1:val, -2], color='green')
#plt.ylim(-2e3, 2e3)
fig.tight_layout()
plt.show()

OverflowError: Exceeded cell block limit (set 'agg.path.chunksize' rcparam)

<Figure size 1152x720 with 2 Axes>

In [19]:
print(pump._cases[val1:val])
#print(pressure[val1:val, 0])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#print(np.sum(pump_velocity[val1:val, 0]))

In [ ]:
from scipy import integrate
def contact_pressure(suction_outer_contact_radius, suction_radius,
                     suction_inner_contact_radius, upper_pressure, lower_pressure,
                     density, viscosity, velocity, suction_seat_tilt, displacement):
    pk = (lower_pressure * ((suction_outer_contact_radius / suction_radius - 1)
                                        / (suction_outer_contact_radius / suction_inner_contact_radius - 1))
                      + upper_pressure * ((1 - suction_inner_contact_radius / suction_radius)
                                          / (1 - suction_inner_contact_radius / suction_outer_contact_radius))
                      - (6.0 * velocity * viscosity * density
                         * (suction_outer_contact_radius**2 - suction_inner_contact_radius**2)
                         * (suction_radius - suction_inner_contact_radius)
                         * (suction_outer_contact_radius - suction_radius))
                      / (((displacement+1e-6)**3)
                         * suction_radius * (suction_outer_contact_radius - suction_inner_contact_radius))
                      )
    for i in range(pk.shape[0]):
        if pk[i] <= 2300:
            pk[i] = 2300
            
    plt.plot(suction_radius, pk)
    plt.show()
    f1 = pk * suction_radius
    

    result = integrate.simps(f1, suction_radius)
    return result

In [ ]:
pk = contact_pressure(0.04, np.linspace(0.03644173,0.04,1000), 0.036, 20.0e5, 20.0e5, 1000, 3.5e-4, 0.0, np.pi/2.0, 0.0)
print(pk)
print((18.0e5 + 20.0e5)/2.0*np.pi*(0.04**2-0.036**2)/4.0)
print(18.0e5*np.pi*(0.036**2)/4.0)
print(20.0e5*np.pi*(0.04**2)/4.0)

In [ ]:
(((0.04/2)**2-(0.03644173/2)**2))*np.pi*20e5

In [ ]:
ra = 0.04/2.0
ri = 0.03644173/2.0
pa = 21e5
pf = 20e5
F = np.pi / 2.0 *(ra**2 * pf + ra*ri*(pa-pf)-ri**2 * pa)

In [ ]:
print(F)